## Process Description

![Dietician](Assets/Simpy_Triage.png)

Simulate a process of people visiting a hospital and sent to ambulatory care (i.e. out-patient department) or to emergency care after triage. People visit the clinic, register at the front desk, then go to triage after which 20 percent are processed in ambulatory care and 80 percent assessed in emergency care.

1. **Entity**: People arriving for treatment
2. **Generator**: Generate arrivals  
3. **Inter-Arrrival Time**: An exponential distribution sampled for patient arrivals  
4. **Activity**: Registration followed by triage followed by assessment in ambulatory care (20 percent) or emergency care (80 percent)
5. **Activity Time**: Exponential distributions
6. **Resources**: Receptionist, nurse, doctor OPD, doctor ER
7. **Queues**: People waiting for registration after arrival, for triage after registration and for assessment after assignment to OPD or ER.
8. **Sink**: Exit after assessment.

Based on: https://youtu.be/QV-pJnKrGuc

### 1. Import packages

Aside from **simpy**, we need **random** to sample from random distributions.

In [1]:
import simpy
import random

### 2. Generate Arrivals

Define the generator for arrivals. This puts patients in the queue for consultation.

**Args**:
- env (Simpy environment) required
- mean_IAT (int) mean Inter-Arrival Time for generating arrivals
- meanCT2register (int) mean Cycle Time for registration
- meanCT2triage (int) mean Cycle Time for triage
- meanCT2assessnOPD (int) mean Cycle Time for assessment in OPD
- meanCT2assessnER (int) mean Cycle Time for assessment in ER
- receptionist (Simpy resource) resource
- nurse (Simpy resource) resource
- doctorOPD (Simpy resource) resource
- doctorER (Simpy resource) resource

**Notes**:
- Has a forever-while loop in which it:
	- Creates a patient ID and sends the patient onward on their journey, invoking the function that generates consultations with the patient's ID.
	- Has one ```yield``` statement to timeout for sampled Inter-Arrival Time, after which it increments patient ID. The ```yield``` with timeout will await the action (i.e. completion of timer), freezing until then and next resuming where it left off.

In [2]:
def generate_arrivals(env, mean_IAT, meanCT2register, meanCT2triage, meanCT2assessnOPD, meanCT2assessnER, 
                        receptionist, nurse, doctorOPD, doctorER):
    # Patient zero
    patient_ID = 0

    while True:
        # Send patient onward 
        generator_process_model = generate_process_model(env, meanCT2register, meanCT2triage, meanCT2assessnOPD, meanCT2assessnER, 
                        receptionist, nurse, doctorOPD, doctorER, patient_ID)
        env.process(generator_process_model)

        # Wait for next arrival
        delta4arrival = expovariate(1.0 / mean_IAT)
        yield env.timeout(delta4arrival)

        # Next patient
        patient_ID += 1

### 3. Generate Action

Define the generator for the process model.

**Args**:
- env (Simpy environment) required
- meanCT2register (int) mean Cycle Time for registration
- meanCT2triage (int) mean Cycle Time for triage
- meanCT2assessnOPD (int) mean Cycle Time for assessment in OPD
- meanCT2assessnER (int) mean Cycle Time for assessment in ER
- receptionist (Simpy resource) resource
- nurse (Simpy resource) resource
- doctorOPD (Simpy resource) resource
- doctorER (Simpy resource) resource
- patient_ID (int) patient ID from the function that generates arrivals

**Notes**: 
- For each activity in the process model, has two ```with``` declarations as follows:
    - Request the resource and ```yield``` until available
    - Execute the activity by timing out for sampled activity time. 
- Records timestamps at: arrived, queued for activity, started activity, exit
- The fork in the road is implemented by conditional logic that samples a uniform random distribution

In [ ]:
def generate_process_model(env, meanCT2register, meanCT2triage, meanCT2assessnOPD, meanCT2assessnER, 
                        receptionist, nurse, doctorOPD, doctorER, patient_ID)
                        